# LangGraph Agent v1 — Basic Q&A

Single-node agent that answers directly with the LLM (no tools).

Quick guide:
- State: messages
- Node: analyze (answer_question) → END
- Observe prints: [ANSWER]
- Run the last cell to see a clean, single execution trace

In [ ]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangGraph
from langgraph.graph import StateGraph, END

# LangChain core
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

In [ ]:
class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="The chat history")
    class Config:
        arbitrary_types_allowed = True

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)

In [ ]:
def answer_question(state: AgentState):
    print("[ANSWER] Executing answer_question node…")

    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledgeable assistant. Today's date is 2025-08-27. Answer concisely and factually. If unsure or information is unavailable, say you don't know. Avoid speculation."),
        ("human", "Question: {q}")
    ])
    ans = llm.invoke(prompt.format_messages(q=user_query)).content.strip()
    return {"messages": [AIMessage(content=ans)]}

In [ ]:
# Build graph
workflow_v1 = StateGraph(AgentState)
workflow_v1.add_node("analyze", answer_question)
workflow_v1.add_edge("analyze", END)

workflow_v1.set_entry_point("analyze")
agent = workflow_v1.compile()

In [ ]:
# Test v1
query = "Who won 2024 presidential elections in US?"
initial_state = AgentState(messages=[HumanMessage(content=query)])
final_state = agent.invoke(initial_state)


In [ ]:
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f"Human: {m.content}")
    elif isinstance(m, AIMessage):
        print(f"AI: {m.content}")
    else:
        print(m)